In [1]:
import numpy as np
import pandas as pd

In [2]:
patients=pd.read_csv('patients.csv')
treatments=pd.read_csv('treatments.csv')
adverse_reactions=pd.read_csv('adverse_reactions.csv')
treatments_cut=pd.read_csv('treatments_cut.csv')

In [3]:
patients_copy=patients.copy()
treatments_copy=treatments.copy()
adverse_reactions_copy=adverse_reactions.copy()
treatments_cut_copy=treatments_cut.copy()

### Quality

[Completeness, Validity, Accuracy, Consistency]

#### `Patients`

- Zip code col is stored in decimal format[V]
- Certain zip codes of 4 digits[V] 
- For the state col sometimes full names are use and other times short forms are used[IC]
- Missing values for contact details of 12 patients[C]
- Incorrect data types for assigned_sex and birthdate cols[?]
- Erroneous data for weights and heights[Accuracy]
- Inconsistent way of storing phone numbers in contact col[IC]
- Duplicate and not useful data in dataset[IC]

#### `Treatments`

- hba1c_change col has missing values[C]
- Novodra and Auralin cols has u attached to numbers[V]
- Only 280 rows are there, instead of 350[C] (349 + 1 Duplicate Joseph Day)
- Name and surname col is in lower case[IC]
- Incorrect data type for auralin and novadra cols[V]
- hba1c_change 9 instead of 4[Accuracy]
- use of dashes instead of Nan in aurolin and novodra cols[IC]

#### `Adverse Reactions`

- Name and surname col is in lower case[IC]


### Tidiness

#### `Patients`

- Phone and email together in one col i.e. contact

#### `Treatments`

- 3 variables stored in auralin and novodra cols

#### `Adverse Reactions`

- No point of using this table

In [4]:
adverse_reactions_copy['given_name'],adverse_reactions_copy['surname']=adverse_reactions_copy['given_name'].str.title(),adverse_reactions_copy['surname'].str.title()

In [5]:
treatments_copy=treatments_copy.append(treatments_cut_copy,ignore_index=True)
treatments_copy.drop_duplicates(['given_name','surname'],inplace=True)
treatments_copy['hba1c_change']=treatments_copy['hba1c_start']-treatments_copy['hba1c_end']
treatments_copy=treatments_copy.melt(id_vars=['given_name','surname','hba1c_start','hba1c_end','hba1c_change'], var_name='treatment', value_name='dose')
treatments_copy=treatments_copy[treatments_copy['dose']!='-']
treatments_copy['min_dose'],treatments_copy['max_dose']=treatments_copy['dose'].str.split('-').str
treatments_copy.drop(columns=['dose'],inplace=True)
treatments_copy['min_dose'],treatments_copy['max_dose']=treatments_copy['min_dose'].str[:-2],treatments_copy['max_dose'].str[:-1]
treatments_copy['given_name'],treatments_copy['surname']=treatments_copy['given_name'].str.title(),treatments_copy['surname'].str.title()
treatments_copy=treatments_copy.merge(adverse_reactions_copy, on=['given_name','surname'], how='left')
treatments_copy=treatments_copy.astype({'treatment':'category', 'min_dose':int, 'max_dose':int})

In [6]:
import re
import requests
from bs4 import BeautifulSoup

patients_copy=patients.copy()
patients_copy.drop_duplicates(['assigned_sex','given_name', 'surname', 'address'], inplace=True)
patients_copy=patients_copy[(patients_copy['given_name']!='John') & (patients_copy['surname']!='Doe')]


webpage=requests.get("https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes").text

soup=BeautifulSoup(webpage, 'lxml')

data=soup.find_all('tr')

d={}

for i in data :
    
    j=i.find_all('td')
    
    if j!=[] :
        d[j[2].text.strip()]=j[0].text.strip()

pattern='[a-zA-Z][a-zA-Z0-9!#$%&*+-//=?^_`{|}~.]+@[a-z]+\.[a-z]+'

patients_copy['state'].replace(d, inplace=True)
patients_copy['state']=patients_copy['state'].str.title()
patients_copy.at[4,'height']=72
patients_copy.at[210,'weight']=127
patients_copy=patients_copy.astype({'assigned_sex':'category', 'contact':str, 'zip_code':str, 'birthdate':'datetime64', 'height':float})

patients_copy['email']=''
patients_copy['phone_number']=''

for i in patients_copy.index.to_list() :
    
    if patients_copy.at[i,'contact'] != 'nan' :
        
        x=re.search(pattern,patients_copy.at[i,'contact'])
        
        patients_copy.at[i,'email']=x.group()
        
        s=x.span()
        
        temp=None
        
        if s[0]==0 :
            temp=patients_copy.at[i,'contact'][s[1]:]
        else :
            temp=patients_copy.at[i,'contact'][:s[0]]
            
        if temp[0] == '+' :
            temp=temp[1:]
            
        if temp[0:2] == '1 ' :
            temp=temp[2:]
            
        if temp[0] == '(' :
            temp=temp[1:4] + '-' + temp[6:]
            
        if len(temp.split()) > 1  or len(temp)==10:
            temp=temp[0:3] + '-' + temp[4:7] + '-' + temp[8:]
            
        patients_copy.at[i,'phone_number']=temp
            
    else :
        patients_copy.at[i,'email']=np.nan
        patients_copy.at[i,'phone_number']=np.nan
        
    if patients_copy.at[i,'zip_code'] != 'nan' :
        
        x=patients_copy.at[i,'zip_code'][:-2]
        
        if len(x) != 5 :
            
            j=len(x)
            while j != 5 :
                x='0'+x
                j+=1
            
        patients_copy.at[i,'zip_code']=x
        
    else :
        patients_copy.at[i,'zip_code']=np.nan
        
patients_copy.drop(columns=['contact'], inplace=True)

In [7]:
patients_copy.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,email,phone_number
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390,United States,1976-07-10,121.7,66.0,19.6,ZoeWellish@superrito.com,951-719-9170
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812,United States,1967-04-03,118.8,66.0,19.2,PamelaSHill@cuvox.de,217-569-3204
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467,United States,1980-02-19,177.8,71.0,24.8,JaeMDebord@gustr.com,402-363-6804
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,New Jersey,07095,United States,1951-07-26,220.9,70.0,31.7,PhanBaLiem@jourrapide.com,732-636-8246
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,Alabama,36303,United States,1928-02-18,192.3,72.0,26.1,TimNeudorf@cuvox.de,334-515-7487


In [13]:
treatments_copy.sample(30)

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,min_dose,max_dose,adverse_reaction
59,Kaodilinakachukwu,Nnonso,7.76,7.40,0.36,auralin,63,68,NaN
166,Joseph,Tucker,7.67,7.30,0.37,auralin,48,56,NaN
325,Miłosław,Wiśniewski,7.51,7.08,0.43,novodra,34,33,injection site discomfort
27,Gabryŝ,Tomaszewski,7.87,7.47,0.40,auralin,29,37,NaN
158,Maryam,Dratchev,9.91,9.46,0.45,auralin,37,45,NaN
336,Daimy,Tromp,9.41,8.94,0.47,novodra,40,45,NaN
51,Xiuxiu,Chang,7.64,7.23,0.41,auralin,32,41,hypoglycemia
18,Władysław,Wieczorek,7.92,7.47,0.45,auralin,24,37,NaN
29,Onyekachukwu,Obinna,7.58,7.12,0.46,auralin,37,46,NaN
54,Liisa,Seppälä,7.87,7.51,0.36,auralin,36,47,NaN
